In [ ]:
!pip install --upgrade bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!pip install transformers datasets evaluate torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, pipeline, BitsAndBytesConfig
from datasets import load_dataset
from evaluate import load as load_metric
import logging
import os
import re

In [ ]:
# Setup logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Define model paths
MODELS = {
    "UI-TARS 7B SFT": "bytedance-research/UI-TARS-7B-SFT",
    "UI-TARS 7B DPO": "bytedance-research/UI-TARS-7B-DPO"
}

In [ ]:
def load_model(model_path):
    """Load model and tokenizer with Hugging Face authentication."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    hf_token = os.getenv("HUGGINGFACE_TOKEN")  # Store your token securely, not hardcoded!

    """Load model and tokenizer."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.empty_cache()  # Clear GPU memory before loading
    model = Qwen2VLForConditionalGeneration.from_pretrained(

        model_path,
        torch_dtype=torch.float16,
        device_map={"": 0},
        offload_folder="offload",  # Save excess weights to disk
        quantization_config=BitsAndBytesConfig(load_in_8bit=True),
        token=hf_token  # Pass token for gated models
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path, token=hf_token)
    logger.info(f"{model_path} Model Loaded Successfully!")
    return model, tokenizer

In [ ]:
def evaluate_qa(model, tokenizer, dataset_name="squad"):
    """Evaluate on SQuAD for QA."""
    dataset = load_dataset(dataset_name, split="validation[:100]")
    metric = load_metric("squad_v2")



    for example in dataset:
        input_text = f"Q: {example['question']} Context: {example['context']} A:"
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        metric.add(predictions={"id": example["id"], "prediction_text": result,  "no_answer_probability": 0.0}, references={"id": example["id"], "answers": example["answers"]})

    return metric.compute()

In [ ]:
def evaluate_math(model, tokenizer, dataset_name="gsm8k"):
    """Evaluate on GSM8K for Math QA."""
    dataset = load_dataset(dataset_name, 'main', split="test[:50]")
    metric = load_metric("accuracy")

    for example in dataset:
        input_text = f"Solve: {example['question']}"
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        answer = str(example['answer']).strip()
        if answer.isdigit():  # Ensure numeric conversion if possible
            answer = int(answer)
        try:
            prediction = float(prediction) if '.' in prediction else int(prediction)
            answer = float(answer) if '.' in answer else int(answer)
        except ValueError:
            prediction, answer = 0, 0  # Default to 0 if conversion fails
        metric.add(prediction=prediction, reference=answer)

    return metric.compute()

In [ ]:
def evaluate_reasoning(model, tokenizer, dataset_name="hellaswag"):
    """Evaluate on HellaSwag for reasoning."""
    dataset = load_dataset(dataset_name, split="validation[:100]")
    metric = load_metric("accuracy")

    for example in dataset:
        input_text = f"{example['ctx']} \n Options: {example['endings']}"
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=50)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        metric.add(prediction=prediction, reference=example['label'])

    return metric.compute()

In [ ]:
def evaluate_summarization(model, tokenizer, dataset_name="cnn_dailymail", subset="3.0.0"):
    """Evaluate on CNN/DailyMail for summarization."""
    dataset = load_dataset(dataset_name, subset, split="test[:50]")
    metric = load_metric("rouge")

    for example in dataset:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs = tokenizer(example['article'], return_tensors="pt", truncation=True).to(device) if torch.cuda.is_available() else tokenizer(example['article'], return_tensors="pt", truncation=True)
        outputs = model.generate(**inputs, max_new_tokens=150)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        metric.add(prediction=summary, reference=example['highlights'])

    return metric.compute()

In [ ]:
# Run benchmark
for model_name, model_path in MODELS.items():
    print(f"\nEvaluating {model_name}...")
    model, tokenizer = load_model(model_path)

    print("SQuAD QA:", evaluate_qa(model, tokenizer))
    print("GSM8K Math:", evaluate_math(model, tokenizer))
    print("HellaSwag Reasoning:", evaluate_reasoning(model, tokenizer))
    print("CNN/DailyMail Summarization:", evaluate_summarization(model, tokenizer))


Evaluating UI-TARS 7B SFT...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

ValueError: invalid literal for int() with base 10: '2.5'